In [ ]:
import tensorflow as tf
import sklearn as sk 
import matplotlib.pyplot as plt
import cv2 as cv 
import keras 
import os 
import pandas as pd
from PIL import Image
import itertools 
from sklearn.datasets import *
from sklearn.metrics import confusion_matrix
from keras.utils import plot_model 
import random 
from keras.layers import *

In [ ]:
%pwd

'd:\\Learning\\Custom'

In [ ]:
train = "./data/imgs/ASL_Alphabet/train/"
test = "./data/imgs/ASL_Alphabet/test/" 
valid = "./data/imgs/ASL_Alphabet/valid/"

In [ ]:
classes = ["del", "nothing", "space"] + [chr(_) for _ in range(ord('A'), ord('Z')+1)]
classes.sort() 
classes 

In [ ]:
%mkdir "./data/imgs/compress"
%mkdir "./data/imgs/compress/train/"
%mkdir "./data/imgs/compress/test/" 
%mkdir "./data/imgs/compress/valid/" 

In [ ]:
base_dir = './data/imgs/compress/' 
for folder in os.listdir(base_dir): 
    path_ = os.path.join(base_dir, folder)
    if "test" not in folder: 
        for char in range(ord('A'), ord('Z')+1):
            folder_name = chr(char) 
            folder_path = os.path.join(path_, folder_name)
            os.makedirs(folder_path, exist_ok=True) 
 
        os.makedirs(path_+"/del", exist_ok=True)
        os.makedirs(path_+"/space", exist_ok=True) 
        os.makedirs(path_+"/nothing", exist_ok=True) 

In [ ]:
def preprocess(input_dir: str, target_size: tuple[int], type: str):
    if "test" not in type: 
        for i in os.listdir(input_dir):
            i_path = os.path.join(input_dir, i)
            for img in os.listdir(i_path): 
                img_path = os.path.join(i_path, img)
                imag = cv.imread(img_path)
                imag = cv.cvtColor(imag,cv.COLOR_BGR2GRAY)
                imag = cv.resize(imag,target_size)
                cv.imwrite(os.path.join(f"./data/imgs/compress/{type}/{i}", img),imag)
                # with Image.open(img_path) as imag: 
                #     grayscale = imag.convert("L") 
                #     resized = grayscale.resize(target_size)
                #     resized.save(os.path.join( 
                #         f"./data/imgs/compress/{type}/{i}", img))
    else: 
        for img in os.listdir(input_dir):
            img_path = os.path.join(input_dir, img)
            imag = cv.imread(img_path)
            imag = cv.cvtColor(imag,cv.COLOR_BGR2GRAY)
            imag = cv.resize(imag,target_size)
            cv.imwrite(os.path.join(f"./data/imgs/compress/{type}", img),imag)

In [ ]:
preprocess(train,(64,64),"train")
preprocess(test,(64,64),"test") 
preprocess(valid,(64,64),"valid")
 

In [ ]:
%mkdir "./data/annotations/compress_gray/"

A subdirectory or file ./data/annotations/compress_gray/ already exists.


In [ ]:
train = "./data/imgs/compress/train/"
test = "./data/imgs/compress/test/" 
valid = "./data/imgs/compress/valid/"

In [ ]:
def annotate(data_dir:str, type:str):
    annotations = [] 
    if "test" not in  type:
        for alphabet_folder in os.listdir(data_dir):
            alphabet_path = os.path.join(data_dir, alphabet_folder)
            for image_file in os.listdir(alphabet_path): 
                image_path = os.path.join(alphabet_path,  image_file)
                label = alphabet_folder 
                with Image.open(image_path) as img:
                    width, height = img.size 
                annotations.append({ 
                    "image_path": image_path,
                    "label": label, 
                    "x_min": 0,   
                    "y_min": 0,   
                    "x_max": width,   
                    "y_max": height   
                }) 
    else: 
        for image_file in os.listdir(data_dir):
                image_path = os.path.join(data_dir, image_file)
                label = image_file.split("_")[0] 
                with Image.open(image_path) as img :
                    width, height = img.size 
                annotations.append({ 
                    "image_path": image_path,
                    "label": label, 
                    "x_min": 0,   
                    "y_min": 0,   
                    "x_max": width,   
                    "y_max": height   
                }) 
    annotations_df  = pd.DataFrame(annotations)
    annotations_df.to_csv(f"./data/annotations/{type}.csv", index=False)
 

In [ ]:
annotate(train,'compress_gray/train')
annotate(test,'compress_gray/test') 
annotate(valid,'compress_gray/valid') 